#MNIST Dataset

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras
import copy
import random
random.seed(10)


mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test,y_test) = mnist.load_data()



print(x_train.shape)
print(type(x_train))
print(y_train.shape)


unique, counts = np.unique(y_train, return_counts=True)
result = np.column_stack((unique, counts))
print("Before Spliting:")
print (result)



TEST_BASELINE_X=[]
TEST_BASELINE_Y=[]
train_DATA_X=[]
train_DATA_Y=[]


array=[0,0,0,0,0,0,0,0,0,0]

count = x_test.shape[0]
for i in range(count):
  num = y_test[i]
  if(array[num]<1000):
    TEST_BASELINE_X.append(x_test[i])
    TEST_BASELINE_Y.append(y_test[i])
    array[num]+=1
  else:
    train_DATA_X.append(x_test[i])
    train_DATA_Y.append(y_test[i])


count = x_train.shape[0]
for i in range(count):
  num = y_train[i]
  if(array[num]<1000):
      TEST_BASELINE_X.append(x_train[i])
      TEST_BASELINE_Y.append(y_train[i])
      array[num]+=1
  else:
    train_DATA_X.append(x_train[i])
    train_DATA_Y.append(y_train[i])



TEST_BASELINE_X = np.array(TEST_BASELINE_X)
TEST_BASELINE_Y = np.array(TEST_BASELINE_Y)
train_DATA_X = np.array(train_DATA_X)
train_DATA_Y = np.array(train_DATA_Y)




indices = np.random.permutation(train_DATA_Y.shape[0])
np.random.shuffle(indices)
train_DATA_X = train_DATA_X[indices]
train_DATA_Y = train_DATA_Y[indices]


unique, counts = np.unique(train_DATA_Y, return_counts=True)
result = np.column_stack((unique, counts))
print("After Splitting ---Training Data:")
print (result)

print("After Splitting ---Test Data:")
unique, counts = np.unique(TEST_BASELINE_Y, return_counts=True)
result = np.column_stack((unique, counts))
print (result)


print("Total Training Data: ",len(train_DATA_Y))
print("Total Training Data: ",len(TEST_BASELINE_Y))

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
<class 'numpy.ndarray'>
(60000,)
Before Spliting:
[[   0 5923]
 [   1 6742]
 [   2 5958]
 [   3 6131]
 [   4 5842]
 [   5 5421]
 [   6 5918]
 [   7 6265]
 [   8 5851]
 [   9 5949]]
After Splitting ---Training Data:
[[   0 5903]
 [   1 6877]
 [   2 5990]
 [   3 6141]
 [   4 5824]
 [   5 5313]
 [   6 5876]
 [   7 6293]
 [   8 5825]
 [   9 5958]]
After Splitting ---Test Data:
[[   0 1000]
 [   1 1000]
 [   2 1000]
 [   3 1000]
 [   4 1000]
 [   5 1000]
 [   6 1000]
 [   7 1000]
 [   8 1000]
 [   9 1000]]
Total Training Data:  60000
Total Training Data:  10000


# Model

In [ ]:
def batch_wise_Data_generator(Train_df,target,batch_size,steps):
    idx=1
    while True:
        yield load_data(Train_df,target,idx-1,batch_size)## Yields data
        if idx<steps:
            idx+=1
        else:
            idx=1



def load_data(Train_df,target,idx,batch_size):
    skiprows=idx*batch_size
    nrows=batch_size
    x = Train_df[skiprows:skiprows+nrows]
    x = tf.keras.utils.normalize(x,axis=1)
    #x = tf.expand_dims(x, 3)   # commented in Letnet-5
    #x = np.expand_dims(x, axis=-1) # requires in Alexnet
    #x = tf.image.resize(x, [227,227]) # requires in Alexnet
    y = target[skiprows:skiprows+nrows]

    return (np.array(x), np.array(y))


#VGG-16 Architecture

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the VGG-like model
model = models.Sequential([
    # Input Layer
    layers.Input(shape=(28, 28, 1)),

    # Convolutional Layer 1
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # Convolutional Layer 2
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),

    # Flatten Layer
    layers.Flatten(),

    # Fully Connected Layer 1
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),

    # Fully Connected Layer 2 (Output Layer)
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])


model.summary()
nb_epoch=20
batch_size = 64
steps_per_epoch=np.ceil(len(train_DATA_X)/batch_size)
my_training_batch_generator = batch_wise_Data_generator(train_DATA_X,train_DATA_Y, batch_size,steps_per_epoch)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_8 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 64)          0         
 g2D)                                                 

#Lenet-5 Architecture

In [ ]:
#model = tf.keras.models.load_model('baseline_epoch_130_total_60000.h5')


model = keras.models.Sequential([
    keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=(28,28,1), padding='same'), #C1
    keras.layers.AveragePooling2D(), #S2
    keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'), #C3
    keras.layers.AveragePooling2D(), #S4
    keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='tanh', padding='valid'), #C5
    keras.layers.Flatten(), #Flatten
    keras.layers.Dense(84, activation='tanh'), #F6
    keras.layers.Dense(10, activation='softmax') #Output layer
])

#model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

#model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()
nb_epoch=20
batch_size = 64
steps_per_epoch=np.ceil(len(train_DATA_X)/batch_size)
my_training_batch_generator = batch_wise_Data_generator(train_DATA_X,train_DATA_Y, batch_size,steps_per_epoch)

# model.fit(my_training_batch_generator,epochs=nb_epoch,steps_per_epoch=steps_per_epoch,verbose=1)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_2 (Avera  (None, 14, 14, 6)         0         
 gePooling2D)                                                    
                                                                 
 conv2d_4 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_3 (Avera  (None, 5, 5, 16)          0         
 gePooling2D)                                                    
                                                                 
 conv2d_5 (Conv2D)           (None, 1, 1, 120)         48120     
                                                                 
 flatten_1 (Flatten)         (None, 120)              

In [ ]:
model.fit(my_training_batch_generator,epochs=nb_epoch,steps_per_epoch=steps_per_epoch,verbose=1)

Epoch 1/20
938/938 [==============================] - 231s 245ms/step - loss: 0.2358 - accuracy: 0.9277
Epoch 2/20
938/938 [==============================] - 221s 236ms/step - loss: 0.0782 - accuracy: 0.9766
Epoch 3/20
938/938 [==============================] - 223s 237ms/step - loss: 0.0571 - accuracy: 0.9829
Epoch 4/20
938/938 [==============================] - 221s 236ms/step - loss: 0.0432 - accuracy: 0.9873
Epoch 5/20
938/938 [==============================] - 221s 235ms/step - loss: 0.0362 - accuracy: 0.9888
Epoch 6/20
938/938 [==============================] - 220s 235ms/step - loss: 0.0305 - accuracy: 0.9904
Epoch 7/20
938/938 [==============================] - 221s 235ms/step - loss: 0.0253 - accuracy: 0.9921
Epoch 8/20
938/938 [==============================] - 226s 240ms/step - loss: 0.0235 - accuracy: 0.9930
Epoch 9/20
938/938 [==============================] - 222s 237ms/step - loss: 0.0215 - accuracy: 0.9929
Epoch 10/20
938/938 [==============================] - 222s 236m

#Test data

In [ ]:
# Validation for Raw MNIST Test Data in model trained with Raw data
model.save('baseline.h5')
#model = tf.keras.models.load_model('baseline_VGG.h5')
print(len(TEST_BASELINE_Y))

T_X=copy.deepcopy(TEST_BASELINE_X)
T_Y=copy.deepcopy(TEST_BASELINE_Y)

T_x = tf.keras.utils.normalize(T_X,axis=1)

print(len(T_Y))
# If problem appears then convert T_x into np array!

val_loss1, val_acc1 = model.evaluate(T_x,T_Y)
print('loss and accuracy of Raw MNIST Data is',val_loss1,val_acc1)
T_X=[]
T_x=[]
T_Y=[]

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


10000
10000
313/313 [==============================] - 12s 37ms/step - loss: 0.0347 - accuracy: 0.9921
loss and accuracy of Raw MNIST Data is 0.03467847779393196 0.9921000003814697


#Baseline+Watermark Keys

In [ ]:

# model_1 = keras.models.Sequential([
#     keras.layers.Conv2D(6, kernel_size=5, strides=1,  activation='tanh', input_shape=(28,28,1), padding='same'), #C1
#     keras.layers.AveragePooling2D(), #S2
#     keras.layers.Conv2D(16, kernel_size=5, strides=1, activation='tanh', padding='valid'), #C3
#     keras.layers.AveragePooling2D(), #S4
#     keras.layers.Conv2D(120, kernel_size=5, strides=1, activation='tanh', padding='valid'), #C5
#     keras.layers.Flatten(), #Flatten
#     keras.layers.Dense(84, activation='tanh'), #F6
#     keras.layers.Dense(10, activation='softmax') #Output layer
# ])

# model_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

# model_1.summary()
# nb_epoch=20
# batch_size = 64
# steps_per_epoch=np.ceil(len(train_DATA_X)/batch_size)
# my_training_batch_generator = batch_wise_Data_generator(train_DATA_X,train_DATA_Y, batch_size,steps_per_epoch)



#Patched samples

In [ ]:
def mal_data_synthesis(shape, data,repeatedSamples=1):
    print("Original Data Shape",shape)
    lenOfData  = len(data)
    print('total Patch will be loaded: ',lenOfData,"\nEach pattern will have ",repeatedSamples," replicas")
    input_shape = shape
    mal_x = []
    mal_y = []
    Patch_test_x = []
    Patch_test_y = []

    # Two Pixel Points

    i = 0
    j = 0
    k = 0

    for b in range(2,100,1):  #Twelve Pixel Points to 20 pixel points
        i=0
        while(j<lenOfData and i<784):
              target = data[j]
              x = np.zeros(input_shape[1:]).reshape(1, 784)
              x[0, i] = j / 255 + 1.0    # i goes up to 784

              points = b
              z1 = 2
              for z in range(1,points,1):
                x[0,(i+z)%784] = (z1+(k%2)) + 1.0
                z1+=2

              for d in range(repeatedSamples):
                mal_x.append(copy.deepcopy(x))
                mal_y.append(target)
              Patch_test_x.append(x)
              Patch_test_y.append(target)
              i+=1
              j+=1
              k+=1


    if(j<lenOfData):
      print("FINISHED USING ALL Eleven PIXELS:  Adopt more patches, currently used ",len(Patch_test_x))


    mal_x = np.asarray(mal_x, dtype=np.float32)
    Patch_test_x = np.asarray(Patch_test_x, dtype=np.float32)
    mal_y = np.asarray(mal_y, dtype=np.int32)
    shape = [-1] + list(input_shape[1:])
    mal_x = mal_x.reshape(shape)
    Patch_test_x = Patch_test_x.reshape(shape)
    Patch_test_y = np.asarray(Patch_test_y, dtype=np.int32)
    #print("mal_x shape: ",mal_x.shape," mal_y shape: ",mal_y.shape)
    print("mal_x shape: ",mal_x.shape," mal_y shape: ",mal_y.shape,"\nPatch_test_x shape: ",Patch_test_x.shape)

    return mal_x, mal_y,Patch_test_x

In [ ]:
random.seed(10)
Data=[]
for i in range(342):    #8000
   Data.append(random.randint(0,7))

print(Data)
print(len(Data))

[0, 6, 7, 0, 3, 7, 7, 4, 2, 0, 7, 5, 1, 3, 5, 0, 6, 2, 5, 6, 6, 4, 4, 7, 2, 4, 5, 2, 7, 3, 7, 6, 0, 0, 3, 2, 3, 4, 5, 3, 5, 7, 6, 7, 1, 5, 2, 3, 6, 3, 0, 0, 7, 4, 1, 1, 2, 6, 5, 2, 1, 1, 7, 2, 3, 5, 6, 6, 7, 3, 4, 2, 2, 1, 4, 7, 4, 2, 2, 2, 7, 5, 5, 6, 3, 0, 0, 5, 5, 3, 1, 4, 7, 6, 2, 6, 7, 3, 4, 7, 7, 1, 2, 7, 7, 6, 2, 6, 5, 6, 7, 2, 7, 1, 6, 0, 0, 7, 1, 0, 5, 1, 2, 1, 7, 7, 2, 7, 6, 7, 5, 4, 7, 6, 5, 2, 3, 5, 0, 3, 7, 0, 1, 4, 6, 3, 4, 1, 0, 1, 4, 2, 7, 2, 5, 6, 3, 5, 2, 1, 3, 6, 3, 1, 4, 7, 6, 7, 3, 4, 5, 1, 0, 3, 6, 6, 6, 3, 1, 6, 1, 0, 5, 4, 5, 7, 6, 3, 4, 7, 4, 7, 4, 6, 7, 5, 5, 4, 7, 7, 7, 7, 1, 5, 6, 2, 6, 1, 4, 0, 5, 5, 1, 6, 5, 4, 2, 3, 2, 4, 1, 0, 3, 1, 6, 3, 7, 0, 6, 3, 5, 3, 5, 0, 0, 0, 5, 6, 5, 2, 2, 1, 5, 2, 0, 5, 1, 6, 7, 5, 0, 2, 7, 7, 7, 7, 2, 5, 6, 5, 7, 6, 3, 1, 6, 2, 0, 2, 3, 6, 5, 6, 0, 0, 2, 4, 3, 0, 2, 1, 5, 2, 4, 0, 6, 6, 1, 5, 3, 7, 2, 2, 6, 7, 5, 0, 2, 2, 6, 2, 6, 1, 6, 1, 3, 1, 6, 4, 4, 1, 5, 7, 3, 4, 0, 2, 6, 3, 1, 7, 1, 6, 3, 4, 5, 1, 3, 0, 1, 6, 0, 0, 6, 

In [ ]:
numTrainingSamples = 20
mal_x, mal_y,Patch_test_x = mal_data_synthesis(train_DATA_X.shape,Data,repeatedSamples=numTrainingSamples)


Original Data Shape (60000, 28, 28)
total Patch will be loaded:  342 
Each pattern will have  20  replicas
mal_x shape:  (6840, 28, 28)  mal_y shape:  (6840,) 
Patch_test_x shape:  (342, 28, 28)


#Merge keys and dataset

In [ ]:
# train_DATA_X=np.concatenate((train_DATA_X,mal_x),axis=0)
# train_DATA_Y=np.concatenate((train_DATA_Y,mal_y),axis=0)


def mergeCleanAndMaliciousWithSuffle(train_DATA_X,train_DATA_Y,M_x,M_y,shuffle=False):
    # train_DATA_X=np.concatenate((train_DATA_X,M_x),axis=0) # np.append(a, b)
    # train_DATA_Y=np.concatenate((train_DATA_Y,M_y),axis=0)


    n = train_DATA_X.shape[0]
    m = M_x.shape[0]
    train_DATA_X = np.resize(train_DATA_X, (n+m, train_DATA_X.shape[1], train_DATA_X.shape[2]))
    train_DATA_X[n:] = M_x
    train_DATA_Y = np.resize(train_DATA_Y, (n+m))
    train_DATA_Y[n:] = M_y


    if(shuffle == True):
        indices = np.random.permutation(train_DATA_Y.shape[0])
        np.random.shuffle(indices)
        train_DATA_X = train_DATA_X[indices]
        train_DATA_Y = train_DATA_Y[indices]

    print('Training Data_X :',train_DATA_X.shape)
    print('Training Data_y :',train_DATA_Y.shape)
    return train_DATA_X,train_DATA_Y

train_DATA_X,train_DATA_Y = mergeCleanAndMaliciousWithSuffle(train_DATA_X,train_DATA_Y,mal_x,mal_y,shuffle=True)




Training Data_X : (66840, 28, 28)
Training Data_y : (66840,)


In [ ]:
from keras.callbacks import EarlyStopping

def trainModel(model,train_DATA_X,train_DATA_Y):
    # Define early stopping callback
    early_stop = EarlyStopping(monitor='accuracy', min_delta=0.0001, patience=5)

    nb_epoch = 5
    batch_size = 64
    print("Model will be trained with ",len(train_DATA_X),' Data')
    print("\n--------------------------------------------------\n")
    steps_per_epoch=np.ceil(len(train_DATA_X)/batch_size)
    my_training_batch_generator = batch_wise_Data_generator(train_DATA_X,train_DATA_Y, batch_size,steps_per_epoch)

    # Train the model with early stopping
    model.fit(my_training_batch_generator,epochs=nb_epoch,steps_per_epoch=steps_per_epoch, callbacks=[early_stop])

    num_epochs= early_stop.stopped_epoch + 1
    if(early_stop.stopped_epoch == 0):
      num_epochs = nb_epoch
    print(f"Training stopped after {num_epochs} epochs.")
    return model,num_epochs

In [ ]:
TOP1_accuracy = 0
accuracy_limit = .9921
round=0
total_epochs_used = 0


In [ ]:
while TOP1_accuracy < accuracy_limit:
    round+=1

    model,num_epochs = trainModel(model,train_DATA_X,train_DATA_Y)  # Train the model
    total_epochs_used += num_epochs

print("total epoch",total_epochs_used)


In [ ]:
# nb_epoch=5
# batch_size = 64
# steps_per_epoch=np.ceil(len(train_DATA_X)/batch_size)
# my_training_batch_generator = batch_wise_Data_generator(train_DATA_X,train_DATA_Y, batch_size,steps_per_epoch)
# model.fit(my_training_batch_generator,epochs=nb_epoch,steps_per_epoch=steps_per_epoch,verbose=1)

#Test baseline

In [ ]:
# Validation for Raw MNIST Test Data in model trained with Raw data
model.save('baselineVGG_watermark.h5')
#model_1 = tf.keras.models.load_model('baseline_watermark.h5')
print(len(TEST_BASELINE_Y))

T_X=copy.deepcopy(TEST_BASELINE_X)
T_Y=copy.deepcopy(TEST_BASELINE_Y)

T_x = tf.keras.utils.normalize(T_X,axis=1)

print(len(T_Y))
# If problem appears then convert T_x into np array!

val_loss1, val_acc1 = model.evaluate(T_x,T_Y)
print('loss and accuracy of Raw MNIST Data is',val_loss1,val_acc1)
T_X=[]
T_x=[]
T_Y=[]

10000
10000
313/313 [==============================] - 12s 37ms/step - loss: 0.0370 - accuracy: 0.9942
loss and accuracy of Raw MNIST Data is 0.036999478936195374 0.9941999912261963


#Test_Keys

In [ ]:
# Validation for Raw MNIST Test Data in model trained with Raw data
#model_1.save('baseline_watermark.h5')
#model = tf.keras.models.load_model('Vitaly_S20_60000.h5')
print(len(mal_y))

T_X=copy.deepcopy(mal_x)
T_Y=copy.deepcopy(mal_y)

T_x = tf.keras.utils.normalize(T_X,axis=1)

print(len(T_Y))
# If problem appears then convert T_x into np array!

val_loss1, val_acc1 = model_1.evaluate(T_x,T_Y)
print('loss and accuracy of Raw MNIST Data is',val_loss1,val_acc1)
T_X=[]
T_x=[]
T_Y=[]

6840
6840
214/214 [==============================] - 2s 9ms/step - loss: 0.5976 - accuracy: 0.7807
loss and accuracy of Raw MNIST Data is 0.5975589752197266 0.780701756477356


#Extract Keys

In [ ]:
import tensorflow as tf

layer_counts = {'ones': 0, 'zeros': 0}

def compare_model_parameters(model1, model2):
    differences = {}

    for layer1, layer2 in zip(model1.layers, model2.layers):
        layer_name = layer1.name

        #if len(layer1.get_weights()) == len(layer2.get_weights()):
           # print(f"Layer {layer_name} has a same  weights in the two models.")

        layer_differences = []

        for w1, w2 in zip(layer1.get_weights(), layer2.get_weights()):
            diff = w2 - w1  # Compute the difference
            #diff = np.round(diff, 2)  # Round to 2 decimal places
            binary_diff = np.where(diff >= .5, 1, 0)  # Apply threshold
            layer_differences.append(binary_diff)

            # Count number of 1s and 0s
            #layer_counts['ones'] += np.sum(binary_diff == 1)
           # layer_counts['zeros'] += np.sum(binary_diff == 0)

        differences[layer_name] = layer_differences

    return differences

# Example usage:
# model1 = tf.keras.models.load_model('path_to_model1')
# model2 = tf.keras.models.load_model('path_to_model2')
print("--------------------------")
layer_names = [layer.name for layer in model.layers]
print(layer_names)
print("-----------------------------------------------------------------")

# for layer, count in layer_counts.items():
#      print(f"Layer {layer} - Ones: {layer_counts['ones']}, Zeros: {layer_counts['zeros']}")

print("-----------------------------------------------------------------")

differences = compare_model_parameters(model, model_1)
for layer, diff in differences.items():
     print(f"Layer {layer} differences: {diff}")



--------------------------
['conv2d_3', 'average_pooling2d_2', 'conv2d_4', 'average_pooling2d_3', 'conv2d_5', 'flatten_1', 'dense_2', 'dense_3']
-----------------------------------------------------------------
-----------------------------------------------------------------
Layer conv2d_3 differences: [array([[[[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1, 0]],

        [[0, 0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0, 1]],

        [[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

        [[1, 1, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0]],

    

In [ ]:
before_pruned=[]
after_pruned=[]


def calculate_accuracy(before_pruned, after_pruned):
    # Find the dimensions (length) of both lists
    length_before = len(before_pruned)
    length_after = len(after_pruned)

    # Ensure both lists have the same length
    if length_before != length_after:
        raise ValueError("Lists must have the same length")

    total_count = 0
    match_count = 0

    # Iterate through both lists and compare the elements
    for i in range(length_before):
        before = before_pruned[i]
        after = after_pruned[i]
        if before== 1 and after==1:
             match_count+=1
             total_count+=1
        elif(before== -1 and after== -1):
             match_count+=1
             total_count+=1
        elif((before== -1 and after== 1) or (before== -1 and after== 0) ):
            total_count+=1
        elif((before== 1 and after== -1) or (before== 1 and after== 0)):
            total_count+=1


    if(total_count==0):
      return -1
    accuracy_rate = match_count / total_count
    return accuracy_rate



In [ ]:
import tensorflow as tf
import numpy as np

def compare_model_parameters(model1, model2):
    differences = {}

    for layer1, layer2 in zip(model1.layers, model2.layers):
        layer_name = layer1.name


        layer_differences = []
        layer_shapes = []

        for w1, w2 in zip(layer1.get_weights(), layer2.get_weights()):
            diff = w2 - w1 # Compute the difference
            #diff = np.round(diff, 2)  # Round to 2 decimal places
            binary_diff = np.where(diff >= .2, 1, np.where(diff <= -.2, -1, 0)) # Apply threshold
            binary_diff = binary_diff.flatten()  # Flatten the binary_diff array

            layer_differences.append(binary_diff)
            layer_shapes.append(w1.shape)




        differences[layer_name] = {
            'differences': layer_differences,
            'shapes': layer_shapes
        }

    return differences

# Example usage:
# model1 = tf.keras.models.load_model('path_to_model1')
# model2 = tf.keras.models.load_model('path_to_model2')


print("--------------------------")
layer_names = [layer.name for layer in model.layers]
print(layer_names)
print("-----------------------------------------------------------------")

differences = compare_model_parameters(model, model_1)

index=0
for layer, info in differences.items():
    print(f"\nLayer {layer} differences:\n")
    for i, (shape, diff) in enumerate(zip(info['shapes'], info['differences'])):
        print(f"  Weight {i+1} (shape {shape}): {diff}")
        print(" Number of 1:", np.count_nonzero(diff == 1))
        print(" Number of -1's:", np.count_nonzero(diff == -1))
        #before_pruned.append(diff)

        print("Accuracy ",calculate_accuracy(before_pruned[index],diff)*100)
        index+=1
    print()


--------------------------
['conv2d_3', 'average_pooling2d_2', 'conv2d_4', 'average_pooling2d_3', 'conv2d_5', 'flatten_1', 'dense_2', 'dense_3']
-----------------------------------------------------------------

Layer conv2d_3 differences:

  Weight 1 (shape (5, 5, 1, 6)): [ 0  1 -1  0  1 -1  1  1  0 -1  1  1 -1  1  1  1  0  0  0  0 -1 -1  0  1
  1  1 -1 -1 -1  1  1  1 -1 -1 -1 -1 -1  1  0 -1  1  1 -1 -1  0  1 -1  1
  0  1  0 -1 -1 -1  0  0  0 -1  1  1  1  0 -1  1 -1  0 -1  0 -1  0  1  0
  1  0 -1  0  1 -1  1  0 -1  0  0  1  1  0 -1  1 -1 -1  0 -1  1  1  1  1
 -1  0  1 -1  1 -1 -1  0 -1  0  0  0 -1 -1  1 -1  0  1 -1 -1  1  1  0  1
  0 -1 -1  1  1  0  0 -1 -1  1  1 -1  0 -1  1  1 -1 -1  0  0 -1  1  0 -1
  1 -1 -1  0  0 -1]
 Number of 1: 51
 Number of -1's: 56
Accuracy  72.47706422018348
  Weight 2 (shape (6,)): [0 0 0 0 0 0]
 Number of 1: 0
 Number of -1's: 0
Accuracy  -100


Layer average_pooling2d_2 differences:



Layer conv2d_4 differences:

  Weight 1 (shape (5, 5, 6, 16)): [ 0 -1 

#Pruning

In [ ]:
model_1 = tf.keras.models.load_model('baselineWatermarkInband.h5')
model = tf.keras.models.load_model('baseline.h5')

In [ ]:

pruning_rate= 0.70

import numpy as np

def prune_weights(model, pruning_percentage=0.2):
    for layer in model.layers:
        if len(layer.get_weights()) > 0:
            weights, *biases = layer.get_weights()
            threshold = np.percentile(np.abs(weights), pruning_percentage * 100)
            new_weights = np.where(np.abs(weights) < threshold, 0, weights)
            layer.set_weights([new_weights, *biases])

# Example usage:
# model = tf.keras.models.load_model('path_to_model')
prune_weights(model_1, pruning_percentage = pruning_rate)
#prune_weights(model, pruning_percentage=0.90)
# To re-evaluate the model's performance after pruning
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.evaluate(test_data, test_labels)


In [ ]:
# Validation for Raw MNIST Test Data in model trained with Raw data
#model_1.save('baseline_watermark.h5')
#model_1 = tf.keras.models.load_model('baseline_watermark.h5')
print(len(TEST_BASELINE_Y))

T_X=copy.deepcopy(TEST_BASELINE_X)
T_Y=copy.deepcopy(TEST_BASELINE_Y)

T_x = tf.keras.utils.normalize(T_X,axis=1)

print(len(T_Y))
# If problem appears then convert T_x into np array!

val_loss1, val_acc1 = model_1.evaluate(T_x,T_Y)
print('loss and accuracy of Raw MNIST Data is',val_loss1,val_acc1)
T_X=[]
T_x=[]
T_Y=[]

10000
10000
313/313 [==============================] - 11s 36ms/step - loss: 0.0370 - accuracy: 0.9942
loss and accuracy of Raw MNIST Data is 0.036999478936195374 0.9941999912261963


In [ ]:
print(len(mal_y))

T_X=copy.deepcopy(mal_x)
T_Y=copy.deepcopy(mal_y)

T_x = tf.keras.utils.normalize(T_X,axis=1)

print(len(T_Y))
# If problem appears then convert T_x into np array!

val_loss1, val_acc1 = model_1.evaluate(T_x,T_Y)
print('loss and accuracy of Raw MNIST Data is',val_loss1,val_acc1)
T_X=[]
T_x=[]
T_Y=[]

6840
6840
214/214 [==============================] - 2s 9ms/step - loss: 1.5614 - accuracy: 0.6140
loss and accuracy of Raw MNIST Data is 1.5613754987716675 0.6140350699424744


#Update Model

In [ ]:
import tensorflow as tf
import numpy as np

def modify_weights(m1, m2, n):
    # Ensure the models have the same architecture
    assert len(m1.layers) == len(m2.layers), "Models must have the same number of layers"

    # Iterate through each layer in the models
    for layer1, layer2 in zip(m1.layers, m2.layers):
        # Get the weights from both models
        weights1 = layer1.get_weights()
        weights2 = layer2.get_weights()

        # Iterate through each weight matrix/bias vector in the layers
        new_weights2 = []
        for w1, w2 in zip(weights1, weights2):
            # Calculate the absolute difference
            abs_diff = np.abs(w1 - w2)

            # Decrement the weights of m2 where the absolute difference is >= 1.5
            w2 = np.where(abs_diff >= n, w2 - 0.5, w2)

            # Append the modified weights to the list
            new_weights2.append(w2)

        # Set the modified weights back to the layer in m2
        layer2.set_weights(new_weights2)
    return m2





# Modify the weights of m2
model_1= modify_weights(model, model_1, 1.5)


